# Capstone Battle of Neighborhoods (Week 2)

## 1)Business Problem

In this project we will try to find an optimal location(s) for a bakery in Downtown Toronto, Canada.

We will try to detect neighborhoods with :
* no/least frequency of bakeries
* no/least frequency of bread related shops eg: bagel shop, donut shop, and sandwich place
* bakery is not top 5 venues in the neighborhood

We will use our data to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## 2)Data Processing

Based on definition of our problem, factors that will influence our decision are:
* number of existing bread related shops in the neighborhood
* number of bakeries in the neighborhood, if any
* rank of bakery among venues existing in the neighborhood

Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**


## 3)Methodology

In this project we will direct our efforts on detecting areas of Toronto that have low bread-related shops, particularly those with low number of bakery. 

The steps are:
* Data gathering and cleanup on Toronto neighborhoods dataset
* Exploration of Downtown Toronto: get top 100 venues within 200 meters
* Analyze frequency of bread-related shops in neighborhoods of Downtown Toronto
* Find ranking of bakery among venues in Downtown Toronto neighborhood

After working on the steps above, we should be able to determine the optimal location(s) to open a bakery in Downtown Toronto, which is the central spot in Toronto.

## 4)Results

#### A)Data gathering and cleanup

Import all necessary libaries

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Saved the wiki table as csv file,uploaded the csv to my github, used the link to download it to jupyterlab

Filter out "Not assigned" Boroughs

In [8]:
#download the table from my github
filename = "https://gist.githubusercontent.com/dioo94/19fc42019b168a80e4f65ed336cb8610/raw/13f19891e28bc67ed4293b2c6ea4470b1bc297a4/toronto_wiki.csv"
#load the csv file
df = pd.read_csv(filename)
df.head()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


set "not assigned" neighbourhood to be name of borough

In [9]:
#set "not assigned" neighbourhood to be name of borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


combine neighbourhoods with same postcode and borough

In [10]:
#combine neighbourhoods with same postcode and borough
df=df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ','.join(x)).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Saved the coordinates table as csv file,uploaded the csv to my github, used the link to download it to jupyterlab

In [11]:
#download the table from my github
filename = "https://raw.githubusercontent.com/dioo94/Coursera_Capstone/master/Geospatial_Coordinates.csv"
#load the csv file
df_ll = pd.read_csv(filename)
df_ll = df_ll.rename(columns={'Postal Code': 'Postcode'})
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


merge neighbourhood table with coordinate table

In [12]:
neighborhoods=pd.merge(df,df_ll,on='Postcode')
neighborhoods.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


#### B)Exploration of Downtown Toronto

Next is to cluster only the neighborhoods in Downtown Toronto. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto data.

In [15]:
dtor_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtor_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [16]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.65145205, -79.3620078930752.


Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = '1NPATOI0XXVNPXMBO2OLI2X24ATQQXR5XFUQ24HBCNWOVB3F' # your Foursquare ID
CLIENT_SECRET = 'C5CWCUPUQG0EYUXKY540UENIGFZVC42VPUBP2AKNR5TD2K40' # your Foursquare Secret
VERSION = '20190216' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1NPATOI0XXVNPXMBO2OLI2X24ATQQXR5XFUQ24HBCNWOVB3F
CLIENT_SECRET:C5CWCUPUQG0EYUXKY540UENIGFZVC42VPUBP2AKNR5TD2K40


#### Let's explore the first neighborhood in our dataframe.

In [18]:
dtor_data.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

In [19]:
neighborhood_latitude = dtor_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude =dtor_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dtor_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


Now, let's get the top 100 venues that are in Harbourfront,Regent Park within a radius of 200 meters.

In [34]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 200 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=1NPATOI0XXVNPXMBO2OLI2X24ATQQXR5XFUQ24HBCNWOVB3F&client_secret=C5CWCUPUQG0EYUXKY540UENIGFZVC42VPUBP2AKNR5TD2K40&v=20190216&ll=43.6542599,-79.3606359&radius=200&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6997b94434b957cf74f949'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.6560599018, 'lng': -79.35815270005945},
   'sw': {'lat': 43.6524598982, 'lng': -79.36311909994056}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(50)

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
5,Enoch Turner Schoolhouse,History Museum,43.652873,-79.361672
6,Massimo's Kitchen Studio,Italian Restaurant,43.654770,-79.359698


#### C)Frequency of bread-related shops in Downtown Toronto

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
# type your answer here

dtor_venues = getNearbyVenues(names=dtor_data['Neighbourhood'],
                                   latitudes=dtor_data['Latitude'],
                                   longitudes=dtor_data['Longitude']
                                  )



Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [54]:
# one hot encoding
dtor_onehot = pd.get_dummies(dtor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtor_onehot['Neighborhood'] = dtor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dtor_onehot.columns[-1]] + list(dtor_onehot.columns[:-1])
dtor_onehot = dtor_onehot[fixed_columns]

dtor_onehot.head(20)

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [55]:
dtor_grouped = dtor_onehot.groupby('Neighborhood').mean().reset_index()
dtor_grouped_bread = dtor_grouped[['Neighborhood','Bakery', 'Bagel Shop', 'Donut Shop', 'Sandwich Place']]
dtor_grouped_bread

,Neighborhood,Bakery,Bagel Shop,Donut Shop,Sandwich Place
0,"Adelaide,King,Richmond",0.030000,0.000000,0.000000,0.000000
1,Berczy Park,0.035714,0.017857,0.000000,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown,St. James Town",0.041667,0.000000,0.000000,0.020833
4,Central Bay Street,0.012195,0.000000,0.012195,0.024390
5,"Chinatown,Grange Park,Kensington Market",0.040000,0.010000,0.010000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.000000,0.000000,0.000000,0.000000
8,"Commerce Court,Victoria Hotel",0.030000,0.000000,0.000000,0.010000
9,"Design Exchange,Toronto Dominion Centre",0.010000,0.000000,0.000000,0.010000


#### D)Bakery ranking among venues

In [57]:
num_top_venues = 10

for hood in dtor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dtor_grouped[dtor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04
5                  Bar  0.03
6                  Gym  0.03
7           Restaurant  0.03
8               Bakery  0.03
9     Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4              Bakery  0.04
5         Cheese Shop  0.04
6  Seafood Restaurant  0.04
7      Farmers Market  0.04
8                Café  0.04
9            Beer Bar  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0     Airport Service  0.14
1    Airport Terminal  0.14
2      Airport Lounge  0.14
3               Plane  0.07
4       Boat or Ferry  0.07
5    Sculpture Garden  0.07
6     Harbor 

## 5)Discussions

Our analysis shows that overall, neighborhoods in Downtown Toronto do not have high number of bakery or any bread-related shops available. Indeed, there are neighborhoods with relatively high frequency of bakery, however it's only a few of them. One can notice that there is a disparity in distribution of bakery in Downtown Toronto. There are even neighborhoods with 0 frequency of bakery such as CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara,Christie,Church, Wellesley, and Rosedale. These locations can be the contenders for a new place to open the bakery.

What is more interesting among the places with 0 frequency of bakery, they do not have any other bread-related shops such as bagel shop, donut shop, and sandwich shop as well. So from subtitution goods competition stand point, these places do not have any existing competition. Another advantage is the new place can get the so-called first-mover advantage by opening a new bakery in one of those places. 

Based on the ranking of venues among the aforementioned neighborhoods above, we believe that Christie, Church, and Wellesley are the top 3 neighborhoods for the new bakery location. This is because based on our venue ranking, we see that these places already have a well established food and beverages venues which can be an indicator of high consumer traffic within those areas. As bakery is a type of consumer good that is usually categorized as a complementary good, we think putting a bakery in areas with a busy traffic of people trying to get food from restaurants or cafe is a sensible strategy.

Nonetheless, the final decision is depending on the stakeholders and without a say, further studies of different factors should be considered in the final decision. Third party or expert opinion on those particular areas should be considered as well. 

## 6)Conclusion

Purpose of this project was to identify Toronto areas close to center with low number of bread-related shops (particularly Downtown Toronto) in order to aid stakeholders in narrowing down the search for optimal location for a new bakery. From our analysis we came up with top 3 neighborhoods such as Chritie, Church, and Wellesley that should be considered for the new bakery. This is for reasons such as non-existing bread-related shops and potential consumer traffic within the areas. 

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration various additional factors.